In [ ]:
# Example - https://github.com/cornelliusyudhawijaya/Churn_Causality_Analysis/blob/main/Causal%20Analysis%20Do%20Why.ipynb
    
import numpy as np
import pandas as pd

from dowhy import CausalModel
import dowhy.datasets 

# Avoid printing dataconversion warnings from sklearn and numpy
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('../../../data/student/Student.csv')

## Drop categorical features
data = data.drop(['school','address', 'famsize', 'Pstatus','Mjob', 'Fjob', 'reason', 'guardian'],axis=1)

## Drop NULL values
data = data.dropna()

## Change symbolics to numerics
data['sex'] = np.where(data['sex'] == 'M', 1, 0)
data['schoolsup'] = np.where(data['schoolsup'] == 'yes', 1, 0)
data['famsup'] = np.where(data['famsup'] == 'yes', 1, 0)
data['paid'] = np.where(data['paid'] == 'yes', 1, 0)
data['activities'] = np.where(data['activities'] == 'yes', 1, 0)
data['nursery'] = np.where(data['nursery'] == 'yes', 1, 0)
data['higher'] = np.where(data['higher'] == 'yes', 1, 0)
data['internet'] = np.where(data['internet'] == 'yes', 1, 0)
data['romantic'] = np.where(data['romantic'] == 'yes', 1, 0)
data['Probability'] = np.where(data['Probability'] > 12, 1, 0)


protected_attribute = 'sex'

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data = pd.DataFrame(scaler.fit_transform(data),columns = data.columns)

print(data.head(5))

In [ ]:
causal_graph = """
digraph {
sex;
age;
studytime;
health;
Probability;
U[label="Unobserved Confounders"];
sex -> Probability;
sex -> studytime; studytime -> Probability;
age -> Probability;
sex -> health; health -> Probability;
U->sex;U->age;U->Probability;
}
"""

## dowhy works for binary inputs only


data['Probability'] = np.where(data['Probability'] == 0, False, True)
data['sex'] = np.where(data['sex'] == 1, True, False)
data['race'] = np.where(data['race'] == 1, True, False)

In [ ]:
# With graph
model=CausalModel(
        data = data,
        treatment="sex",
        outcome="Probability",
        graph=causal_graph.replace("\n", " ")
        )

In [ ]:
model.view_model()

from IPython.display import Image, display
display(Image(filename="causal_model.png"))

In [ ]:
identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
print(identified_estimand)

In [ ]:
causal_estimate = model.estimate_effect(identified_estimand,
        method_name="backdoor.propensity_score_stratification")
print(causal_estimate)
print("Causal Estimate is " + str(causal_estimate.value))

In [ ]:
# Causal effect on the control group (ATC)
causal_estimate_att = model.estimate_effect(identified_estimand,
        method_name="backdoor.propensity_score_stratification",
        target_units = "atc")
print(causal_estimate_att)
print("Causal Estimate is " + str(causal_estimate_att.value))

# Add Random Common Cause

In [ ]:
refutel = model.refute_estimate(identified_estimand,causal_estimate, "random_common_cause")
print(refutel)

# Replace Treatment with Placebo

In [ ]:
refutel = model.refute_estimate(identified_estimand,causal_estimate,method_name="placebo_treatment_refuter",placebo_type="permute")
print(refutel)

# Remove Random Subset of Data

In [ ]:
refutel = model.refute_estimate(identified_estimand,causal_estimate, "data_subset_refuter")
print(refutel)